## **Mount Drive**

In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [2]:
cd /content/gdrive/MyDrive/VIT/Tamil Argumentation

/content/gdrive/MyDrive/VIT/Tamil Argumentation


## **Install**

In [3]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 58.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.9 MB/s eta 0:00:00


## **Import Libraries**

In [5]:
import pandas as pd
import numpy as np

from tqdm import tqdm

from copy import deepcopy

from sklearn import metrics
from sklearn.model_selection import KFold

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

from transformers import AutoTokenizer,AutoModel

## **Import Dataset**

In [6]:
df = pd.read_excel("/content/gdrive/MyDrive/VIT/Tamil Argumentation/Twitter Comment Dataset.xlsx")

In [7]:
df.head()

,S No,Tweet,Date of Tweet,Topic,Parent Tweet,Language,Quality,Stance,Argument,Comment,Responding to Tone,Discussing Writer Characteristics,Remark,Relevancy
0,1,"Bro imagine today is Friday , big star movie i...",2018-05-22,Jalikattu,"And tamil people, jalikattu maadu for money an...",ENGLISH,Med,Undetermined,0,1,0,0,0,Relevant
1,2,Dei unnoda akkarai TN mela not on others and w...,2018-05-22,Jalikattu,"And tamil people, jalikattu maadu for money an...",ENGLISH,Med,Against,0,1,0,1,0,Relevant
2,3,En ninga ivara matum mention panuringa naraiya...,2018-05-22,Jalikattu,"And tamil people, jalikattu maadu for money an...",CODE-MIXED,Med,For,0,1,0,0,0,Relevant
3,4,What is happening in Thoothukudi is totally no...,2018-05-22,Jalikattu,"And tamil people, jalikattu maadu for money an...",ENGLISH,High,Against,1,1,0,0,0,Relevant
4,5,Ungaluku Sterlite protest prachanaya illa Bala...,2018-05-22,Jalikattu,"And tamil people, jalikattu maadu for money an...",CODE-MIXED,Med,Undetermined,0,0,1,0,0,Relevant


In [8]:
df.columns

Index(['S No', 'Tweet', 'Date of Tweet', 'Topic', 'Parent Tweet', 'Language',
       'Quality', 'Stance', 'Argument', 'Comment', 'Responding to Tone',
       'Discussing Writer Characteristics', 'Remark', 'Relevancy'],
      dtype='object')

## **Load Text and Labels**

In [38]:
text = df["Tweet"].to_numpy()

Quality_label = df["Quality"].to_numpy()
Argument_label = df["Argument"].to_numpy()
Comment_label = df["Comment"].to_numpy()
Writer_label = df["Discussing Writer Characteristics"].to_numpy()
Tone_label = df["Responding to Tone"].to_numpy()
Remark_label = df["Remark"].to_numpy()
Relevancy_label = df["Relevancy"].to_numpy()

## **Label Encoding**

In [39]:
encode_dict_quality = {
    "High": np.array([1, 0, 0]),
    "Med": np.array([0, 1, 0]),
    "Low": np.array([0, 0, 1]),
}

encode_dict = {
    1: np.array([1, 0]),
    0: np.array([0, 1]),
}

encode_dict_relevancy = {
    "Relevant": np.array([1, 0]),
    "Irrelevant": np.array([0, 1]),
}

In [40]:
Quality_label = np.array([encode_dict_quality[label] for label in Quality_label])
Argument_label = np.array([encode_dict[label] for label in Argument_label])
Comment_label = np.array([encode_dict[label] for label in Comment_label])
Writer_label = np.array([encode_dict[label] for label in Writer_label])
Tone_label = np.array([encode_dict[label] for label in Tone_label])
Remark_label = np.array([encode_dict[label] for label in Remark_label])
Relevancy_label = np.array([encode_dict_relevancy[label] for label in Relevancy_label])

## **Pre-Config for mBERT**

In [14]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

MAX_LEN = np.max([len(x) for x in text])
MAX_LEN = np.min([MAX_LEN, 510])

BATCH_SIZE = 32
LEARNING_RATE = 1e-1

In [15]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')

## **Build Dataset for mBERT**

In [16]:
class ModelDataset(Dataset):
    def __init__(self, X, y, tokenizer, max_len):
        self.max_len = max_len
        self.text = X
        self.tokenizer = tokenizer
        self.targets = y

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, index):
        text = self.text[index]
        inputs = self.tokenizer.encode_plus(
            text,
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True
        )

        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]

        return {
            'ids': torch.tensor(ids, dtype=torch.long).to(device),
            'mask': torch.tensor(mask, dtype=torch.long).to(device),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long).to(device),
            'targets': torch.tensor(self.targets[index], dtype=torch.float).to(device)
        }

## **Build Model**

In [25]:
class CustomModel(nn.Module):

    def __init__(self):
        super(CustomModel, self).__init__()
        self.bert = AutoModel.from_pretrained('bert-base-multilingual-cased')

        for param in self.bert.parameters():
            param.requires_grad = False

        self.out_layer = nn.Linear(768, 2)

    def forward(self, ids, mask, token_type_ids):
        _, features = self.bert(
            ids, token_type_ids=token_type_ids,
            attention_mask=mask, return_dict=False
        )

        output = self.out_layer(features)

        return output

## **Train Model**

In [28]:
def train(epoch, model, train_loader, loss_fn, optimizer):

    model.train()

    for batch in tqdm(train_loader):

        optimizer.zero_grad()

        ids = batch['ids'].to(device, dtype = torch.long)
        mask = batch['mask'].to(device, dtype = torch.long)
        token_type_ids = batch['token_type_ids'].to(device, dtype = torch.long)
        targets = batch['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        loss = loss_fn(outputs, targets)

        loss.backward()
        optimizer.step()


    print(f'Epoch: {epoch + 1}, Loss:  {loss.item()}')

In [21]:
def validation(data_loader, model):

    model.eval()
    targets = []
    outputs = []

    with torch.no_grad():

        for batch in data_loader:

            ids = batch['ids'].to(device, dtype = torch.long)
            mask = batch['mask'].to(device, dtype = torch.long)
            token_type_ids = batch['token_type_ids'].to(device, dtype = torch.long)
            batch_targets = batch['targets'].to(device, dtype = torch.float)

            batch_outputs = model(ids, mask, token_type_ids)

            targets.extend(batch_targets.cpu().numpy().tolist())
            outputs.extend(batch_outputs.cpu().numpy().tolist())

    return outputs, targets

In [ ]:
kf = KFold(n_splits=5)

model_targets = []
model_labels = []

model = CustomModel().to(device)

loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(params=model.parameters(), lr=LEARNING_RATE)

for train_index, test_index in kf.split(text):

    text_train, text_test = text[train_index], text[test_index]
    labels_train, labels_test = Argument_label[train_index], Argument_label[test_index]

    train_data = ModelDataset(text_train, labels_train, tokenizer, MAX_LEN)
    test_data = ModelDataset(text_test, labels_test, tokenizer, MAX_LEN)

    train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=BATCH_SIZE)

    best_score = -np.inf
    best_weights = None

    EPOCHS = 10

    torch.save(model.state_dict(),"/content/gdrive/MyDrive/VIT/Tamil Argumentation/twitter_model_argument.pth")

    for epoch in range(EPOCHS):

        train(epoch, model, train_loader, loss_fn, optimizer)
        outputs, targets = validation(test_loader, model)

        print(outputs)
        print(" ")
        print(targets)

        targets = [np.argmax(x) for x in targets]
        outputs = [np.argmax(x) for x in outputs]

        score = metrics.f1_score(targets, outputs, average='weighted')

        if score > best_score:
            best_score = score
            best_weights = deepcopy(model.state_dict())

    model.load_state_dict(best_weights)
    torch.save(model.state_dict(),"/content/gdrive/MyDrive/VIT/Tamil Argumentation/twitter_model_argument.pth")

    outputs, targets = validation(test_loader, model)

    targets = [np.argmax(x) for x in targets]
    outputs = [np.argmax(x) for x in outputs]

    model_targets.extend(targets)
    model_labels.extend(outputs)

100%|██████████| 34/34 [00:18<00:00,  1.82it/s]


Epoch: 1, Loss:  0.5598838329315186
[[-11.443680763244629, 4.448367595672607], [-9.194916725158691, 2.7795374393463135], [-7.511484146118164, 2.795736312866211], [-12.234318733215332, 2.9090819358825684], [-8.684109687805176, 3.2519283294677734], [-9.600546836853027, 4.335109233856201], [-7.974612712860107, 3.8351988792419434], [-6.274366855621338, 2.065030097961426], [-10.968130111694336, 4.022838592529297], [-4.135931491851807, 1.4554136991500854], [-5.609196186065674, 1.6326847076416016], [-9.741717338562012, 3.8901522159576416], [-6.422377109527588, 1.494336724281311], [-5.254456520080566, 2.4498343467712402], [-7.857550144195557, 2.30241322517395], [-10.316399574279785, 4.053957939147949], [-9.103894233703613, 2.6927921772003174], [-11.049166679382324, 3.418760061264038], [-8.265060424804688, 2.5099010467529297], [-5.919986248016357, 0.9028308987617493], [-7.503967761993408, 1.2414355278015137], [-6.409285068511963, 1.1343861818313599], [-5.334238529205322, 2.594231128692627], [-7

100%|██████████| 34/34 [00:19<00:00,  1.79it/s]


Epoch: 2, Loss:  0.5588838458061218
[[-9.800026893615723, 4.449311256408691], [-6.9437737464904785, 1.6909692287445068], [-6.132838726043701, 2.7661452293395996], [-9.440315246582031, 2.5434517860412598], [-7.560821056365967, 3.51845645904541], [-8.671320915222168, 3.978992223739624], [-7.38712739944458, 3.615267753601074], [-5.075843811035156, 1.8433669805526733], [-8.299527168273926, 3.781998634338379], [-2.7879297733306885, 0.9509804248809814], [-4.382823467254639, 1.3518425226211548], [-8.403045654296875, 3.5106163024902344], [-4.757795810699463, 0.8465433120727539], [-4.466906547546387, 2.123065710067749], [-5.911749362945557, 1.5707836151123047], [-8.064680099487305, 3.316025733947754], [-6.399147033691406, 1.7224806547164917], [-8.985733032226562, 2.825735092163086], [-5.169861316680908, 1.3945595026016235], [-3.4710543155670166, 0.19029748439788818], [-5.125954627990723, 0.2636212110519409], [-3.065272569656372, -0.21738514304161072], [-6.353850841522217, 3.7421247959136963], [

100%|██████████| 34/34 [00:19<00:00,  1.75it/s]


...

100%|██████████| 34/34 [00:19<00:00,  1.71it/s]


Epoch: 9, Loss:  0.2753322720527649
[[-3.264810562133789, 1.310197353363037], [-6.276706695556641, 5.266434192657471], [-5.768356800079346, 2.1234121322631836], [-7.533563613891602, 5.636188983917236], [-5.340643405914307, 0.837058961391449], [-5.030045509338379, 3.8836967945098877], [-5.357246398925781, 3.495502233505249], [-8.405131340026855, 6.967571258544922], [-2.4460971355438232, 0.658360481262207], [-7.348694324493408, 5.040103435516357], [-2.788900852203369, 1.9851444959640503], [-2.8459320068359375, 1.5061891078948975], [-5.6531758308410645, 3.7194695472717285], [4.737088680267334, -1.268601894378662], [-4.71522855758667, 2.507739782333374], [-6.141822338104248, 2.9206137657165527], [-6.165186405181885, 4.083194255828857], [-6.511630535125732, 4.86740255355835], [-5.8380045890808105, 4.083959102630615], [-8.028007507324219, 5.651363849639893], [-2.4045398235321045, 1.775900959968567], [-6.927393913269043, 4.2962236404418945], [-7.411491870880127, 4.63777494430542], [-3.7305784

100%|██████████| 34/34 [00:19<00:00,  1.72it/s]


Epoch: 10, Loss:  0.36270850896835327
[[0.34174108505249023, 0.8763284683227539], [-3.437394857406616, 4.228213787078857], [-3.803628921508789, 4.501104831695557], [-4.49505615234375, 5.246577739715576], [-4.594691753387451, 5.233916282653809], [-1.8291003704071045, 2.837033987045288], [-2.1563708782196045, 3.0202407836914062], [-5.632519245147705, 6.760752201080322], [-1.3169186115264893, 2.506457805633545], [-3.7237131595611572, 4.449387073516846], [-4.795305252075195, 7.115102291107178], [0.6542525291442871, 0.4857829213142395], [-1.9801124334335327, 2.794398546218872], [-8.335450172424316, 8.73595142364502], [-1.7371554374694824, 2.3122737407684326], [-3.9538681507110596, 5.16686487197876], [-3.9140188694000244, 4.732583522796631], [-3.774807929992676, 4.3923563957214355], [-3.221428871154785, 3.917921304702759], [-5.478630065917969, 6.197798252105713], [-0.3499978184700012, 1.1922776699066162], [-3.2025299072265625, 4.206900119781494], [-3.747981309890747, 4.278140544891357], [-0.

 65%|██████▍   | 22/34 [00:12<00:07,  1.66it/s]

In [ ]:
accuracy = metrics.accuracy_score(model_targets, model_labels)
f1_score_w_avg = metrics.f1_score(model_targets, model_labels, average='weighted')

print(f"Accuracy Score = {accuracy}")
print(f"F1 Score (Weighted) = {f1_score_w_avg}")

print(metrics.classification_report(model_targets, model_labels))

Accuracy Score = 0.6718518518518518
F1 Score (Weighted) = 0.660670534429505
              precision    recall  f1-score   support

           0       0.57      0.37      0.44       197
           1       0.73      0.81      0.77       849
           2       0.53      0.48      0.50       304

    accuracy                           0.67      1350
   macro avg       0.61      0.55      0.57      1350
weighted avg       0.66      0.67      0.66      1350

